In [1]:
%matplotlib inline

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader

import cv2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

import time
import os
import copy
import tarfile

from urllib.parse import urlparse
from urllib.request import urlretrieve

from PIL import Image
from tqdm import tqdm

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

#import albumentations as A
from skimage import filters
from scipy import ndimage as ndi
from skimage import segmentation
from skimage import morphology
from skimage.restoration import inpaint

cudnn.benchmark = True
plt.ion()   # interactive mode

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#helper function
def get_label(row):
    for c in labels.columns:
        if row[c]==1:
            return c

labels = pd.read_csv('ISIC_2019_Training_GroundTruth.csv')
labels['l']=labels.apply(get_label, axis=1)
X=labels.drop(['l','UNK'], axis=1)
y = labels['l']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.21, random_state=42,stratify=y)
X_train['image'] = 'ISIC_2019_Training_Input/ISIC_2019_Training_Input/'+X_train['image']+ '.jpg'
X_test['image'] = 'ISIC_2019_Training_Input/ISIC_2019_Training_Input/'+X_test['image']+ '.jpg'
X_test.to_csv('validation.csv', index=False)
X_train.to_csv('training.csv', index=False)
train_path = 'training.csv'
class_path = 'validation_class.csv'
eval_path = 'validation.csv'
train_img_path = 'ISIC_2019_Training_Input/ISIC_2019_Training_Input'

In [ ]:
labels

In [3]:
class_names = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']

def read_object_labels_csv(file,phase, header=True):
    img_t=[]
    if phase == 'Train':
      dir = 'training.csv'
    elif phase == "Valid":
      dir = 'validation.csv'
    with open(dir, "r") as f:
        csvReader = csv.reader(f)
        next(csvReader, None)
        k=0
        for line in csvReader:
            k+=1
            image_name= line[0]
            label = line[1:9]
            label = torch.from_numpy(np.array(label, dtype=np.float64))
            item = (image_name,label)
            img_t.append(item)
    return img_t


class isic19(Dataset):
    def __init__(self, root, phase, csv_path,transform=None):
        self.root = os.path.abspath(root)
        self.phase = phase
        self.transform = transform
        self.path_images = self.root
        file_csv = pd.read_csv(csv_path)
        self.classes = class_names
        self.images = read_object_labels_csv(file_csv,phase)
        print('[dataset] classification phase={} number of classes={}  number of images={}'.format(phase, len(self.classes), len(self.images)))

    def __getitem__(self, index):
        filename, target = self.images[index]
        filename = filename
        img = Image.open(filename).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        data = {'image':img, 'target': target}
        return data

    def __len__(self):
        return len(self.images)

    def get_number_classes(self):
        return len(self.classes)


In [4]:
class Hair_removal(object):
    
    def __init__(self):
        self.image = None
        
    def __call__(self, image):
        #image = cv2.imread('../input/isic-2019-zip/ISIC_2019_Training_Input/ISIC_2019_Training_Input/ISIC_0000001.jpg')

        image = np.array(image.getdata()).reshape(image.size[0], image.size[1], 3)
        image = image.astype('uint8')
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        resize = cv2.resize(rgb, (224,224), interpolation = cv2.INTER_AREA)

        img = resize[:,:,0] * 0.3 + resize[:,:,1]*0.59 + resize[:,:,2]*0.11
        img = img.astype(np.uint8)

        CLAHE = A.augmentations.transforms.CLAHE()
        contrast = CLAHE(image = img)
        contrast = contrast['image']

        average = cv2.boxFilter(img, -1, (5,5), normalize = True)

        init_hair_mask = contrast - average

        ret,thresh2 = cv2.threshold(init_hair_mask,50,1,
                                    cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        opened = morphology.remove_small_objects(ar = thresh2, min_size=50, connectivity = 1)

        image_result = inpaint.inpaint_biharmonic(resize, opened, multichannel = True)
        image_result = image_result*255
        image_result = image_result.astype('uint8')
        
        ret = Image.fromarray(image_result)
        return ret

In [5]:
from skimage.segmentation import watershed
class Segment(object):
    
    def __init__(self):
        self.image = None
        
    def __call__(self, image):
        #image = cv2.imread('../input/isic-2019-zip/ISIC_2019_Training_Input/ISIC_2019_Training_Input/ISIC_0000001.jpg')

        image = np.array(image.getdata()).reshape(image.size[0], image.size[1], 3)
        image = image.astype('uint8')
        new_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        new_gray = new_gray/255

        elevation_map = filters.sobel(new_gray)

        markers = np.zeros_like(new_gray)
        threshold = filters.threshold_isodata(new_gray)
        markers[new_gray > threshold] = 1
        markers[new_gray < threshold] = 2

        segmented_img = watershed(elevation_map, markers=markers, connectivity=1, offset=None, mask=None)

        segmented_img = ndi.binary_fill_holes(segmented_img - 1)

        segmented_img = morphology.remove_small_objects(segmented_img, min_size=800)

        img_border_cleared = segmentation.clear_border(segmented_img)

        labeled_img = morphology.label(img_border_cleared)

        res = image * segmented_img.reshape(224,224,1)

        ret = Image.fromarray(res)
        return ret

In [23]:
# import required module
import os
# assign directory
directory = 'ISIC_2019_Training_Input/ISIC_2019_Training_Input'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    try:

        f = os.path.join(directory, filename)
        # checking if it is a file
        image = cv2.imread(f)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        resize = cv2.resize(rgb, (224,224), interpolation = cv2.INTER_AREA)
        img = resize[:,:,0] * 0.3 + resize[:,:,1]*0.59 + resize[:,:,2]*0.11
        img = img.astype(np.uint8)
        CLAHE = A.augmentations.transforms.CLAHE()
        contrast = CLAHE(image = img)
        contrast = contrast['image']
        average = cv2.boxFilter(img, -1, (5,5), normalize = True)
        init_hair_mask = contrast - average
        ret,thresh2 = cv2.threshold(init_hair_mask,50,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        opened = morphology.remove_small_objects(ar = thresh2, min_size=50, connectivity = 1)
        image_result = inpaint.inpaint_biharmonic(resize, opened, multichannel = True)
        image_result = image_result*255
        image_result = image_result.astype('uint8')
        finalname = 'preprocessed-hair/'+filename
        cv2.imwrite(finalname, image_result)


        new_gray = cv2.cvtColor(image_result, cv2.COLOR_RGB2GRAY)
        new_gray = new_gray/255

        elevation_map = filters.sobel(new_gray)

        markers = np.zeros_like(new_gray)
        threshold = filters.threshold_isodata(new_gray)
        markers[new_gray > threshold] = 1
        markers[new_gray < threshold] = 2

        segmented_img = watershed(elevation_map, markers=markers, connectivity=1, offset=None, mask=None)

        segmented_img = ndi.binary_fill_holes(segmented_img - 1)

        segmented_img = morphology.remove_small_objects(segmented_img, min_size=800)

        img_border_cleared = segmentation.clear_border(segmented_img)

        labeled_img = morphology.label(img_border_cleared)

        res = image_result * segmented_img.reshape(224,224,1)
        finalname = 'preprocessed-segment/'+filename
        cv2.imwrite(finalname, res)
    except:
        print("Sorry ! You are dividing by zero ")

/tmp/ipykernel_68/462086244.py:25: FutureWarning: `multichannel` is a deprecated argument name for `inpaint_biharmonic`. It will be removed in version 1.0. Please use `channel_axis` instead.
  image_result = inpaint.inpaint_biharmonic(resize, opened, multichannel = True)


Sorry ! You are dividing by zero 
Sorry ! You are dividing by zero 


In [ ]:
plt.title("After segementation and cropping")
plt.imshow(res)
plt.show()

In [ ]:
##checking all fiels
import os
# assign directory
directory = 'ISIC_2019_Training_Input/ISIC_2019_Training_Input'
directory1 = 'preprocessed-hair'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory1, filename)
    if os.path.isfile(f):
        continue
    else:
        print (f)
    

NameError: name 's' is not defined